load libraries

In [14]:
%load_ext autoreload
%autoreload 2
from synthpop.census_helpers import Census
from synthpop import categorizer as cat
from synthpop import synthesizer
from synthpop.recipes import starter
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', 500)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


api key for census data

In [15]:
c = Census('7a25a7624075d46f112113d33106b6648f42686a')

Identify the fields we want at tract and block-group level

In [16]:
#Households
income_columns = ['B19001_0%02dE'%i for i in range(1, 18)]
vehicle_columns = ['B08201_0%02dE'%i for i in range(1, 7)]
workers_columns = ['B08202_0%02dE'%i for i in range(1, 6)]
families_columns = ['B11001_001E', 'B11001_002E']
# year_built_columns= ['B25034_001E', 'B25034_002E', 'B25034_003E'] 
# includes vacant structures?
tenure_columns=['B25063_001E', 'B25075_001E']
block_group_columns = income_columns + families_columns + tenure_columns
tract_columns = vehicle_columns + workers_columns

#Persons
population = ['B01001_001E']
sex = ['B01001_002E', 'B01001_026E']
# race = ['B02001_0%02dE'%i for i in range(1,11)]
male_age_columns = ['B01001_0%02dE'%i for i in range(3,26)]
female_age_columns = ['B01001_0%02dE'%i for i in range(27,50)]
# worker_class_columns=['B08128_0%02dE'%i for i in range(1, 7)]
worker_class_columns=['B24080_0%02dE'%i for i in [1,3,6,7,8,9,10,13,16,17,18,19,20]]
# One more column needed for the 16 and under not incuded in the worker_class population
all_columns = population + sex + male_age_columns + female_age_columns 
# +worker_class_columns

identify which counties we want data for

In [17]:
state='25'
counties=['017', '025']

For each county, get the person and HH data for this tract and all the BGs in the tract

In [18]:
h_acs=pd.DataFrame()
p_acs=pd.DataFrame()
for county in counties:
    print(county)
    h_acs_c = c.block_group_and_tract_query(block_group_columns,
                    tract_columns, state, county, 
                    merge_columns=['tract', 'county', 'state'],
                    block_group_size_attr="B11001_001E",
                    tract_size_attr="B08201_001E",
                    tract=None, year=2017)
    h_acs=pd.concat([h_acs, h_acs_c])
    p_acs_c = c.block_group_query(all_columns, state, county, tract=None, year=2017)
    p_acs=pd.concat([p_acs, p_acs_c])
h_acs=h_acs.reset_index(drop=True)
p_acs=p_acs.reset_index(drop=True)


017
025


In [19]:
# add another category to the worker_class for under-16s not included
# p_acs['B24080_not_employed']=p_acs.apply(lambda row: row['B01001_001E']-row['B24080_001E'], axis=1)

add pumas

In [20]:
h_acs['puma'] = h_acs.apply(lambda row: c.tract_to_puma(row['state'], row['county'], row['tract'])[0], axis=1)
p_acs['puma'] = p_acs.apply(lambda row: c.tract_to_puma(row['state'], row['county'], row['tract'])[0], axis=1)

Create the categorised versions

In [21]:
# Households
h_acs_cat = cat.categorize(h_acs, {
#     ("households", "total"): "B11001_001E",
    ("children", "yes"): "B11001_002E",
    ("children", "no"): "B11001_001E - B11001_002E",
    ("income", "lt35"): "B19001_002E + B19001_003E + B19001_004E + "
                        "B19001_005E + B19001_006E + B19001_007E",
    ("income", "gt35-lt100"): "B19001_008E + B19001_009E + "
                        "B19001_010E + B19001_011E + B19001_012E"
                        "+ B19001_013E",
    ("income", "gt100"): "B19001_014E + B19001_015E + B19001_016E"
                        "+ B19001_017E",
    ("cars", "none"): "B08201_002E",
    ("cars", "one"): "B08201_003E",
    ("cars", "two or more"): "B08201_004E + B08201_005E + B08201_006E",
    ("workers", "none"): "B08202_002E",
    ("workers", "one"): "B08202_003E",
    ("workers", "two or more"): "B08202_004E + B08202_005E",
    ("tenure", "owned"): "B25063_001E",
    ("tenure", "rented"): "B25075_001E"
}, index_cols=['NAME'])
h_acs_cat.head()

cat_name                                           cars                   \
cat_value                                          none  one two or more   
NAME                                                                       
Block Group 4, Census Tract 3686, Middlesex Cou...   76  348         276   
Block Group 3, Census Tract 3686, Middlesex Cou...   71  325         258   
Block Group 1, Census Tract 3686, Middlesex Cou...   31  144         114   
Block Group 2, Census Tract 3686, Middlesex Cou...   25  115          90   
Block Group 5, Census Tract 3686, Middlesex Cou...   60  276         219   

cat_name                                           children      income  \
cat_value                                                no  yes  gt100   
NAME                                                                      
Block Group 4, Census Tract 3686, Middlesex Cou...      271  432    268   
Block Group 3, Census Tract 3686, Middlesex Cou...      453  204    168   
Block Group 1, Census Tract 3686, Middlesex Cou...      140  151    105   
Block Group 2, Census Tract 3686, Middlesex Cou...      143   91    109   
Block Group 5, Census Tract 3686, Middlesex Cou...      326  232    237   

cat_name                                                           tenure  \
cat_value                                          gt35-lt100 lt35  owned   
NAME                                                                        
Block Group 4, Census Tract 3686, Middlesex Cou...        337   98    370   
Block Group 3, Census Tract 3686, Middlesex Cou...        160  329    477   
Block Group 1, Census Tract 3686, Middlesex Cou...        163   23    192   
Block Group 2, Census Tract 3686, Middlesex Cou...         82   43    157   
Block Group 5, Census Tract 3686, Middlesex Cou...        195  126    359   

cat_name                                                  workers       \
cat_value                                          rented    none  one   
NAME                                                                     
Block Group 4, Census Tract 3686, Middlesex Cou...    333     119  288   
Block Group 3, Census Tract 3686, Middlesex Cou...    180     111  269   
Block Group 1, Census Tract 3686, Middlesex Cou...     99      49  119   
Block Group 2, Census Tract 3686, Middlesex Cou...     77      39   95   
Block Group 5, Census Tract 3686, Middlesex Cou...    199      95  228   

cat_name                                                        
cat_value                                          two or more  
NAME                                                            
Block Group 4, Census Tract 3686, Middlesex Cou...         294  
Block Group 3, Census Tract 3686, Middlesex Cou...         275  
Block Group 1, Census Tract 3686, Middlesex Cou...         122  
Block Group 2, Census Tract 3686, Middlesex Cou...          97  
Block Group 5, Census Tract 3686, Middlesex Cou...         233

In [22]:
# Persons
p_acs_cat = cat.categorize(p_acs, {
#     ("population", "total"): "B01001_001E",
    ("age", "19 and under"): "B01001_003E + B01001_004E + B01001_005E + "
                             "B01001_006E + B01001_007E + B01001_027E + "
                             "B01001_028E + B01001_029E + B01001_030E + "
                             "B01001_031E",
    ("age", "20 to 35"): "B01001_008E + B01001_009E + B01001_010E + "
                         "B01001_011E + B01001_012E + B01001_032E + "
                         "B01001_033E + B01001_034E + B01001_035E + "
                         "B01001_036E",
    ("age", "35 to 60"): "B01001_013E + B01001_014E + B01001_015E + "
                         "B01001_016E + B01001_017E + B01001_037E + "
                         "B01001_038E + B01001_039E + B01001_040E + "
                         "B01001_041E",
    ("age", "above 60"): "B01001_018E + B01001_019E + B01001_020E + "
                         "B01001_021E + B01001_022E + B01001_023E + "
                         "B01001_024E + B01001_025E + B01001_042E + "
                         "B01001_043E + B01001_044E + B01001_045E + "
                         "B01001_046E + B01001_047E + B01001_048E + "
                         "B01001_049E", 
#     ("race", "white"):   "B02001_002E",
#     ("race", "black"):   "B02001_003E",
#     ("race", "asian"):   "B02001_005E",
#     ("race", "other"):   "B02001_004E + B02001_006E + B02001_007E + "
#                          "B02001_008E",
    ("sex", "male"):     "B01001_002E",
    ("sex", "female"):   "B01001_026E",
#     ("worker_class", "private_for_profit"): "B24080_003E+ B24080_013E",
#     ("worker_class", "private_non_profit"): "B24080_006E+ B24080_016E",
#     ("worker_class", "government"): "B24080_007E+ B24080_008E + B24080_009E"
#                                     "B24080_017E+ B24080_018E + B24080_019E",
#     ("worker_class", "self_employed"): "B24080_010E+ B24080_020E",
#     ("worker_class", "not_employed"): "B24080_not_employed + B24080_011E + B24080_021E"
    
}, index_cols=['NAME'])
p_acs_cat.head()

cat_name                                                    age           \
cat_value                                          19 and under 20 to 35   
NAME                                                                       
Block Group 4, Census Tract 3686, Middlesex Cou...          453      538   
Block Group 3, Census Tract 3686, Middlesex Cou...          250      657   
Block Group 1, Census Tract 3686, Middlesex Cou...          132      242   
Block Group 3, Census Tract 3834, Middlesex Cou...          521      212   
Block Group 2, Census Tract 3834, Middlesex Cou...          346      403   

cat_name                                                                sex  \
cat_value                                          35 to 60 above 60 female   
NAME                                                                          
Block Group 4, Census Tract 3686, Middlesex Cou...      606      250    856   
Block Group 3, Census Tract 3686, Middlesex Cou...      260      104    660   
Block Group 1, Census Tract 3686, Middlesex Cou...      235       40    316   
Block Group 3, Census Tract 3834, Middlesex Cou...      731      132    658   
Block Group 2, Census Tract 3834, Middlesex Cou...      646      250    998   

cat_name                                                 
cat_value                                          male  
NAME                                                     
Block Group 4, Census Tract 3686, Middlesex Cou...  991  
Block Group 3, Census Tract 3686, Middlesex Cou...  611  
Block Group 1, Census Tract 3686, Middlesex Cou...  333  
Block Group 3, Census Tract 3834, Middlesex Cou...  938  
Block Group 2, Census Tract 3834, Middlesex Cou...  647

Define the pums data we want and some functions for converting to the same categories as the ACS data

In [23]:
h_acs=h_acs.set_index('NAME')
p_acs=p_acs.set_index('NAME')

In [24]:
#Person
def age_cat(r):
    if r.AGEP <= 19: return "19 and under"
    elif r.AGEP <= 35: return "20 to 35"
    elif r.AGEP <= 60: return "35 to 60"
    return "above 60"

def race_cat(r):
    if r.RAC1P == 1: return "white"
    elif r.RAC1P == 2: return "black"
    elif r.RAC1P == 6: return "asian"
    return "other"

def sex_cat(r):
    if r.SEX == 1: return "male"
    return "female"

def tenure_cat(r):
    if r.TEN ==1 or r.TEN ==2: return "owned"
    elif r.TEN ==3:  return "rented"
    return "other"

#Household
def cars_cat(r):
    if r.VEH == 0: return "none"
    elif r.VEH == 1: return "one"
    return "two or more"

def children_cat(r):
    if r.NOC > 0: return "yes"
    return "no"

def income_cat(r): # the example file used FINCP (family income) but the ACS control is Household income
    if r.HINCP > 100000: return "gt100"
    elif r.HINCP > 35000: return "gt35-lt100"
    return "lt35"

def workers_cat(r):
    if r.WIF == 3: return "two or more"
    elif r.WIF == 2: return "two or more"
    elif r.WIF == 1: return "one"
    return "none"

In [25]:
h_pums_cols = ('serialno', 'RT', 'PUMA00', 'PUMA10',  'NP',
                            'TYPE', 'VEH', 'WIF', 'NOC', 'TEN', 'RNTP', 'BDSP', 'YBL', 'HINCP')
p_pums_cols = ('serialno', 'RT', 'PUMA00', 'PUMA10', 'AGEP', 'RAC1P', 'SEX', 'SCHL', 'COW')
all_pumas=list(set(h_acs['puma']))

do the synthesis one PUMA at a time

In [26]:
all_households=pd.DataFrame()
all_persons=pd.DataFrame()

for puma in all_pumas:
    print(puma)
    # get the block groups in this puma
    this_puma_ind=[i for i in range(len(h_acs)) if h_acs.iloc[i]['puma']==puma]
    #download the pums data
    p_pums=c.download_population_pums(state, puma10=puma, usecols=p_pums_cols)
    h_pums=c.download_household_pums(state, puma10=puma, usecols=h_pums_cols)
    #get the joint distribution of pums data
    h_pums, jd_households = cat.joint_distribution(h_pums,
        cat.category_combinations(h_acs_cat.columns),
        {"cars": cars_cat, "children": children_cat, 
         "income": income_cat, "workers": workers_cat, "tenure": tenure_cat})
    p_pums, jd_persons = cat.joint_distribution(
        p_pums,
        cat.category_combinations(p_acs_cat.columns),
        {"age": age_cat, "sex": sex_cat}
    )
    # simulate households and persons for eacg person in each block-group of this PUMA
    for bg_ind in this_puma_ind:
        zone_name=h_acs_cat.index[bg_ind]
        print(zone_name)
        geoid=state+ h_acs.loc[zone_name,'county']+h_acs.loc[zone_name,'tract']+h_acs.loc[zone_name,'block group']
        print(geoid)
        best_households, best_people, people_chisq, people_p= synthesizer.synthesize(h_acs_cat.iloc[bg_ind].transpose(), p_acs_cat.iloc[bg_ind].transpose(), jd_households, jd_persons, h_pums, p_pums,
                   marginal_zero_sub=.01, jd_zero_sub=.001, hh_index_start=0)
    #     add the puma and bg id to each HH
        best_households['NAME']=zone_name
        best_households['home_geoid']=geoid
    #     add people and HH to global list
        all_households=pd.concat([all_households, best_households])
        all_persons=pd.concat([all_persons, best_people])

03400
Block Group 4, Census Tract 3739, Middlesex County, Massachusetts
250173739004
Drawing 330 households
Block Group 1, Census Tract 3739, Middlesex County, Massachusetts
250173739001
Drawing 916 households
Block Group 3, Census Tract 3739, Middlesex County, Massachusetts
250173739003
Drawing 516 households
Block Group 2, Census Tract 3739, Middlesex County, Massachusetts
250173739002
Drawing 725 households
Block Group 2, Census Tract 3737, Middlesex County, Massachusetts
250173737002
Drawing 375 households
Block Group 1, Census Tract 3737, Middlesex County, Massachusetts
250173737001
Drawing 296 households
Block Group 4, Census Tract 3737, Middlesex County, Massachusetts
250173737004
Drawing 226 households
Block Group 3, Census Tract 3737, Middlesex County, Massachusetts
250173737003
Drawing 382 households
Block Group 1, Census Tract 3743, Middlesex County, Massachusetts
250173743001
Drawing 438 households
Block Group 3, Census Tract 3743, Middlesex County, Massachusetts
2501737430

Block Group 3, Census Tract 3671, Middlesex County, Massachusetts
250173671003
Drawing 256 households
Block Group 4, Census Tract 3671, Middlesex County, Massachusetts
250173671004
Drawing 352 households
Block Group 1, Census Tract 3671, Middlesex County, Massachusetts
250173671001
Drawing 308 households
Block Group 2, Census Tract 3671, Middlesex County, Massachusetts
250173671002
Drawing 366 households
Block Group 2, Census Tract 3861, Middlesex County, Massachusetts
250173861002
Drawing 682 households
Block Group 1, Census Tract 3861, Middlesex County, Massachusetts
250173861001
Drawing 358 households
Block Group 3, Census Tract 3861, Middlesex County, Massachusetts
250173861003
Drawing 437 households
01000
Block Group 4, Census Tract 3342, Middlesex County, Massachusetts
250173342004
Drawing 406 households
Block Group 1, Census Tract 3342, Middlesex County, Massachusetts
250173342001
Drawing 370 households
Block Group 2, Census Tract 3342, Middlesex County, Massachusetts
2501733420

Block Group 2, Census Tract 3154.02, Middlesex County, Massachusetts
250173154022
Drawing 740 households
Block Group 1, Census Tract 3154.02, Middlesex County, Massachusetts
250173154021
Drawing 338 households
Block Group 2, Census Tract 3151, Middlesex County, Massachusetts
250173151002
Drawing 1305 households
Block Group 1, Census Tract 3151, Middlesex County, Massachusetts
250173151001
Drawing 903 households
Block Group 2, Census Tract 3162.02, Middlesex County, Massachusetts
250173162022
Drawing 628 households
Block Group 3, Census Tract 3162.02, Middlesex County, Massachusetts
250173162023
Drawing 280 households
Block Group 1, Census Tract 3162.02, Middlesex County, Massachusetts
250173162021
Drawing 528 households
Block Group 1, Census Tract 3311.02, Middlesex County, Massachusetts
250173311021
Drawing 454 households
Block Group 2, Census Tract 3311.02, Middlesex County, Massachusetts
250173311022
Drawing 777 households
Block Group 4, Census Tract 3311.02, Middlesex County, Massa

Block Group 4, Census Tract 3334, Middlesex County, Massachusetts
250173334004
Drawing 378 households
Block Group 2, Census Tract 3334, Middlesex County, Massachusetts
250173334002
Drawing 543 households
Block Group 5, Census Tract 3334, Middlesex County, Massachusetts
250173334005
Drawing 483 households
Block Group 3, Census Tract 3332, Middlesex County, Massachusetts
250173332003
Drawing 511 households
Block Group 1, Census Tract 3332, Middlesex County, Massachusetts
250173332001
Drawing 638 households
Block Group 2, Census Tract 3332, Middlesex County, Massachusetts
250173332002
Drawing 311 households
Block Group 4, Census Tract 3332, Middlesex County, Massachusetts
250173332004
Drawing 616 households
Block Group 1, Census Tract 3333, Middlesex County, Massachusetts
250173333001
Drawing 413 households
Block Group 2, Census Tract 3333, Middlesex County, Massachusetts
250173333002
Drawing 640 households
Block Group 3, Census Tract 3333, Middlesex County, Massachusetts
250173333003
Dra

Drawing 534 households
Block Group 1, Census Tract 1605.01, Suffolk County, Massachusetts
250251605011
Drawing 603 households
Block Group 4, Census Tract 1605.01, Suffolk County, Massachusetts
250251605014
Drawing 441 households
Block Group 2, Census Tract 1605.01, Suffolk County, Massachusetts
250251605012
Drawing 314 households
Block Group 5, Census Tract 1605.01, Suffolk County, Massachusetts
250251605015
Drawing 309 households
Block Group 3, Census Tract 1605.01, Suffolk County, Massachusetts
250251605013
Drawing 97 households
Block Group 2, Census Tract 1603, Suffolk County, Massachusetts
250251603002
Drawing 221 households
Block Group 1, Census Tract 1603, Suffolk County, Massachusetts
250251603001
Drawing 1095 households
Block Group 1, Census Tract 1702, Suffolk County, Massachusetts
250251702001
Drawing 444 households
Block Group 4, Census Tract 1702, Suffolk County, Massachusetts
250251702004
Drawing 539 households
Block Group 2, Census Tract 1702, Suffolk County, Massachusett

...

Block Group 4, Census Tract 2.02, Suffolk County, Massachusetts
250250002024
Drawing 219 households
Block Group 1, Census Tract 3.01, Suffolk County, Massachusetts
250250003011
Drawing 219 households
Block Group 3, Census Tract 3.01, Suffolk County, Massachusetts
250250003013
Drawing 185 households
Block Group 2, Census Tract 3.01, Suffolk County, Massachusetts
250250003012
Drawing 359 households
Block Group 4, Census Tract 3.01, Suffolk County, Massachusetts
250250003014
Drawing 306 households
Block Group 3, Census Tract 8.02, Suffolk County, Massachusetts
250250008023
Drawing 186 households
Block Group 1, Census Tract 8.02, Suffolk County, Massachusetts
250250008021
Drawing 457 households
Block Group 2, Census Tract 8.02, Suffolk County, Massachusetts
250250008022
Drawing 643 households
Block Group 4, Census Tract 8.02, Suffolk County, Massachusetts
250250008024
Drawing 604 households
Block Group 5, Census Tract 8.02, Suffolk County, Massachusetts
250250008025
Drawing 738 households


Drawing 1044 households
Block Group 2, Census Tract 3413, Middlesex County, Massachusetts
250173413002
Drawing 454 households
Block Group 6, Census Tract 3412, Middlesex County, Massachusetts
250173412006
Drawing 275 households
Block Group 2, Census Tract 3412, Middlesex County, Massachusetts
250173412002
Drawing 393 households
Block Group 5, Census Tract 3412, Middlesex County, Massachusetts
250173412005
Drawing 693 households
Block Group 4, Census Tract 3412, Middlesex County, Massachusetts
250173412004
Drawing 376 households
Block Group 3, Census Tract 3412, Middlesex County, Massachusetts
250173412003
Drawing 318 households
Block Group 1, Census Tract 3412, Middlesex County, Massachusetts
250173412001
Drawing 409 households
Block Group 4, Census Tract 3397, Middlesex County, Massachusetts
250173397004
Drawing 286 households
Block Group 2, Census Tract 3397, Middlesex County, Massachusetts
250173397002
Drawing 513 households
Block Group 3, Census Tract 3397, Middlesex County, Massac

Block Group 3, Census Tract 3173.02, Middlesex County, Massachusetts
250173173023
Drawing 567 households
Block Group 1, Census Tract 3173.02, Middlesex County, Massachusetts
250173173021
Drawing 737 households
Block Group 2, Census Tract 3173.02, Middlesex County, Massachusetts
250173173022
Drawing 1190 households
Block Group 2, Census Tract 3173.01, Middlesex County, Massachusetts
250173173012
Drawing 237 households
Block Group 1, Census Tract 3173.01, Middlesex County, Massachusetts
250173173011
Drawing 956 households
Block Group 2, Census Tract 3182, Middlesex County, Massachusetts
250173182002
Drawing 519 households
Block Group 1, Census Tract 3182, Middlesex County, Massachusetts
250173182001
Drawing 1151 households
Block Group 2, Census Tract 3184, Middlesex County, Massachusetts
250173184002
Drawing 487 households
Block Group 1, Census Tract 3184, Middlesex County, Massachusetts
250173184001
Drawing 901 households
Block Group 2, Census Tract 3141.01, Middlesex County, Massachuse

Block Group 2, Census Tract 3121, Middlesex County, Massachusetts
250173121002
Drawing 309 households
Block Group 1, Census Tract 3121, Middlesex County, Massachusetts
250173121001
Drawing 438 households
Block Group 3, Census Tract 3121, Middlesex County, Massachusetts
250173121003
Drawing 440 households
Block Group 1, Census Tract 3123, Middlesex County, Massachusetts
250173123001
Drawing 501 households
Block Group 2, Census Tract 3123, Middlesex County, Massachusetts
250173123002
Drawing 668 households
Block Group 3, Census Tract 3123, Middlesex County, Massachusetts
250173123003
Drawing 816 households
Block Group 1, Census Tract 3120, Middlesex County, Massachusetts
250173120001
Drawing 406 households
Block Group 2, Census Tract 3120, Middlesex County, Massachusetts
250173120002
Drawing 537 households
Block Group 3, Census Tract 3113, Middlesex County, Massachusetts
250173113003
Drawing 416 households
Block Group 1, Census Tract 3113, Middlesex County, Massachusetts
250173113001
Dra

KeyboardInterrupt: 

In [30]:
# all_households['home_geoid']=all_households.apply(lambda row: state+ h_acs.loc[row['NAME'],'county']+h_acs.loc[row['NAME'],'tract']+h_acs.loc[row['NAME'],'block group'], axis=1)

In [32]:
all_households.to_csv('./Boston/clean/synth_households.csv')
all_persons.to_csv('./Boston/clean/synth_persons.csv')

Prepare the data for the ABM
- take a sample of households
- add work locations based on LODES O_D matrix
- save as json

In [35]:
NUM_HOUSEHOLDS=500

In [37]:
synth_hh_df=all_households.sample(n=NUM_HOUSEHOLDS)
synth_hh_df=synth_hh_df.rename(columns= {'RNTP': 'rent', 'BDSP': 'beds'})
synth_hh_df['built_since_jan2010']=synth_hh_df.apply(lambda row: row['YBL']>=14, axis=1)


all_HH_serial_nos=set(synth_hh_df['serialno'])
synth_persons_df=all_persons.loc[all_persons['serialno'].isin(all_HH_serial_nos)]
synth_persons_df['bach_degree']=synth_persons_df.apply(lambda row: 
    "yes" if row['SCHL']>20 else "no", axis=1)
synth_persons_df['bach_degree']=synth_persons_df.apply(lambda row: 
    "yes" if row['SCHL']>20 else "no", axis=1)

/Users/doorleyr/anaconda/envs/urbanSimEnv/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/doorleyr/anaconda/envs/urbanSimEnv/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Process O-D data

In [39]:
all_zones=set(synth_hh_df['home_geoid'])
OD_PATH='Boston/raw/LODES/ma_od_main_JT00_2015.csv'
od=pd.read_csv(OD_PATH)

od['w_block_group']=od.apply(lambda row: str(row['w_geocode'])[0:12], axis=1)
od['h_block_group']=od.apply(lambda row: str(row['h_geocode'])[0:12], axis=1)
od=od.loc[((od['w_block_group'].isin(all_zones)) & (od['h_block_group'].isin(all_zones)))]
# TODO: sample by age group
# od_long=pd.DataFrame()
# for ag in [1,2,3]:
#     odx=od[['h_block_group', 'w_block_group', 'SA0'+str(ag)]].rename(columns={'SA0'+str(ag):'workers'})
#     odx['age_group']=ag
#     odx=odx[odx['workers']>0]
#     od_long=pd.concat([od_long,odx])

Create house and household objects

In [74]:
base_houses, base_households, base_persons=[],[],[]
house_cols=['puma10','beds', 'rent', 'tenure','built_since_jan2010', 'home_geoid']
household_cols=['HINCP', 'cars','NP',
       'workers', 'tenure', 'children', 'income', 'serialno']
person_cols=['COW', 'bach_degree', 'age', 'sex']
person_cols_hh=['income', 'children', 'workers', 'tenure', 'household_id', 'pop_per_sqmile_home']

for ind, row in synth_hh_df.iterrows():
    house_id=len(base_houses)
    household_id=len(base_households)
    add_house={col: row[col] for col in house_cols}
    add_house['house_id']=house_id
    add_house['household_id']=household_id
    # TODO: get the below from PUMS and ACS data
    pop_per_sq_mile=5000   
    add_house['puma_pop_per_sqmeter']=0.000292
    add_house['puma_med_income']=60000
    add_house['pop_per_sqmile']=pop_per_sq_mile
    add_household={col: row[col] for col in household_cols}
    add_household['house_id']=house_id
    add_household['household_id']=household_id
    add_household['pop_per_sqmile_home']=pop_per_sq_mile
    base_houses.append(add_house)
    base_households.append(add_household)

spawn persons from each household and sample their work_geoid

In [77]:
# for household spawn persons from that household
# TODO: avoided repeated calculations of od_subset- make a crosstab at start
for hh in base_households:
    initial_home_geoid=base_houses[hh['house_id']]['home_geoid']
    od_subset=od.loc[od['h_block_group']==initial_home_geoid]
    total_jobs=sum(od_subset['S000'])
    od_subset['weights']=od_subset.apply(lambda row: float(row['S000'])/float(total_jobs), axis=1)
    # TODO: use num adults, not num persons
    person_rows=synth_persons_df[synth_persons_df['serialno']==hh['serialno']].sample(n=hh['NP'])
    for ind, row in person_rows.iterrows():
        add_person={col: row[col] for col in person_cols}
        for hh_col in person_cols_hh:
            add_person[hh_col]=hh[hh_col]
        add_person['person_id']=len(base_persons)
        # TODO, use O-D matrix for work locations
        add_person['work_geoid']=np.random.choice(list(od_subset['w_block_group'].values), 1,p=list(od_subset['weights']))[0]
        base_persons.append(add_person)

/Users/doorleyr/anaconda/envs/urbanSimEnv/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [80]:
import json
json.dump(base_households, open('./Boston/clean/households_sample.json', 'w'))
json.dump(base_houses, open('./Boston/clean/houses_sample.json', 'w'))
json.dump(base_persons, open('./Boston/clean/persons_sample.json', 'w'))